In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [2]:
returns = pd.read_pickle("./Data/returns.pkl")
returns = returns.iloc[1:]

In [3]:
drop_columns = []

for col in returns.columns:
    if returns[col].isnull().all() == True:
        drop_columns.append(col)
        
returns.drop(columns=drop_columns, inplace=True)

In [4]:
def get_investable(t, n):
    """"Find stocks in investable universe at time t+1
    (stocks in the S&P500 that have prices recorded for the last n days)"""
    
    df_investable = returns.copy(deep = True).sort_index(ascending = False)
    
    #add 1 date to get the test features in investable
    t = t + pd.DateOffset(1)
    n += 1
    
    #if t is now a non-trading day, advance until we reach a valid trading day
    while t not in df_investable.index:
        t = t + pd.DateOffset(1)
    
    t_index = df_investable.index.get_loc(t)
    
    #take n_rows worth of data upto time specified
    df_investable = df_investable.iloc[t_index:t_index + n]
    
    #find all stocks that exist in the S&P at this time period
    investable_universe = []
    for col in df_investable.columns:
        if ~df_investable[col].iloc[:n].isna().any():
            investable_universe.append(col)
        
    df_investable = df_investable[investable_universe]
    
    return df_investable

In [38]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def apply_PCA(inv, k):
    X = inv.iloc[1:, :]
    pca = PCA(n_components = k) 
    inv_scaled = StandardScaler().fit_transform(X)   
    principal_components = pca.fit_transform(inv_scaled)

    df = pd.DataFrame(data = principal_components)
    
    #For explained variance table
    components = pca.components_
    component_explained_var = pca.explained_variance_ratio_ * 100
    
    comp_names = [str(i) for i in range(1, len(component_explained_var) + 1)]

    pca_results = pd.DataFrame(data = component_explained_var, index = comp_names)
    pca_results.columns = ['Explained variance (%)']
    pca_results['Explained variance (%)'] = pca_results['Explained variance (%)'].round(2)
    
    return df#, pca_results

In [40]:
def define_y(inv, stock):
    classifications = (inv[stock] > inv.median(axis=1)).astype("int")
    y = pd.DataFrame(data={stock: classifications}, index=inv.index).iloc[:-1]
    
    return y

In [24]:
def train_test(X, y):
    X_train = X.iloc[1:, :]
    X_test = X.iloc[0:1, :]
    y_train = y.iloc[1:]
    y_test = y.iloc[0:1]
    
    return X_train, y_train, X_test, y_test

In [28]:
def model_fit(X_train, y_train):
    model = sklearn.linear_model.LogisticRegression()
    model.fit(X_train, y_train)
    
    return model

In [29]:
def model_predict(model, X_test):
    yhat = model.predict(X_test)
    
    return yhat

In [44]:
def predict_returns(t, n, k):
    inv = get_investable(t, n)
    X = apply_PCA(inv, k)
    
    returns_t = pd.DataFrame(index = inv.columns, columns = ['Pred', 'Actual'])
    
    for stock in inv.columns:
        y = define_y(inv, stock)
        X_train, y_train, X_test, y_test = train_test(X, y)
        model = model_fit(X_train, y_train)
        yhat = model_predict(model, X_test)[0]
        returns_t['Pred'].loc[stock] = yhat
        returns_t['Actual'].loc[stock] = y_test.values[0][0]
    
    return returns_t

In [51]:
def rank_stocks(returns):
    topn = returns[returns["Pred"] == 1]
    botn = returns[returns["Pred"] == 0]
    
    return topn, botn

In [52]:
def portfolio_return(topn, botn):
    return_t = topn['Actual'].mean() - botn['Actual'].mean()
    
    return return_t

In [56]:
def pipeline(n, k, num_stocks):

    time_range = returns.loc['2007':'2021'].index
    
    portfolio = pd.DataFrame(index = time_range, columns = ['Portfolio Return'])
    
    count = 0
    for t in time_range[:-1]:
        pred_actual = predict_returns(t, n, k)
        topn, botn = rank_stocks(pred_actual)
        return_t = portfolio_return(topn, botn)
        t_index = time_range.get_loc(t) + 1
        portfolio['Portfolio Return'].loc[time_range[t_index]] = return_t
        
        count +=1
        print(f'{(count/len(time_range))*100:.2f}% complete')
    
    portfolio['Portfolio Return'] = portfolio['Portfolio Return'].astype('float')
    
    return portfolio

In [57]:
portfolio = pipeline(200, 20, 5)

0.03% complete
0.06% complete
0.08% complete
0.11% complete
0.14% complete
0.17% complete
0.19% complete
0.22% complete
0.25% complete
0.28% complete
0.30% complete
0.33% complete
0.36% complete
0.39% complete
0.41% complete
0.44% complete
0.47% complete
0.50% complete
0.52% complete
0.55% complete
0.58% complete
0.61% complete
0.63% complete
0.66% complete
0.69% complete
0.72% complete
0.74% complete
0.77% complete
0.80% complete
0.83% complete
0.85% complete
0.88% complete
0.91% complete
0.94% complete
0.96% complete
0.99% complete
1.02% complete
1.05% complete
1.08% complete
1.10% complete
1.13% complete
1.16% complete
1.19% complete
1.21% complete
1.24% complete
1.27% complete
1.30% complete
1.32% complete
1.35% complete
1.38% complete
1.41% complete
1.43% complete
1.46% complete
1.49% complete
1.52% complete
1.54% complete
1.57% complete
1.60% complete
1.63% complete
1.65% complete
1.68% complete
1.71% complete
1.74% complete
1.76% complete
1.79% complete
1.82% complete
1.85% comp

14.78% complete
14.81% complete
14.83% complete
14.86% complete
14.89% complete
14.92% complete
14.94% complete
14.97% complete
15.00% complete
15.03% complete
15.05% complete
15.08% complete
15.11% complete
15.14% complete
15.16% complete
15.19% complete
15.22% complete
15.25% complete
15.27% complete
15.30% complete
15.33% complete
15.36% complete
15.38% complete
15.41% complete
15.44% complete
15.47% complete
15.49% complete
15.52% complete
15.55% complete
15.58% complete
15.61% complete
15.63% complete
15.66% complete
15.69% complete
15.72% complete
15.74% complete
15.77% complete
15.80% complete
15.83% complete
15.85% complete
15.88% complete
15.91% complete
15.94% complete
15.96% complete
15.99% complete
16.02% complete
16.05% complete
16.07% complete
16.10% complete
16.13% complete
16.16% complete
16.18% complete
16.21% complete
16.24% complete
16.27% complete
16.29% complete
16.32% complete
16.35% complete
16.38% complete
16.40% complete
16.43% complete
16.46% complete
16.49% c

28.92% complete
28.95% complete
28.98% complete
29.00% complete
29.03% complete
29.06% complete
29.09% complete
29.11% complete
29.14% complete
29.17% complete
29.20% complete
29.23% complete
29.25% complete
29.28% complete
29.31% complete
29.34% complete
29.36% complete
29.39% complete
29.42% complete
29.45% complete
29.47% complete
29.50% complete
29.53% complete
29.56% complete
29.58% complete
29.61% complete
29.64% complete
29.67% complete
29.69% complete
29.72% complete
29.75% complete
29.78% complete
29.80% complete
29.83% complete
29.86% complete
29.89% complete
29.91% complete
29.94% complete
29.97% complete
30.00% complete
30.02% complete
30.05% complete
30.08% complete
30.11% complete
30.14% complete
30.16% complete
30.19% complete
30.22% complete
30.25% complete
30.27% complete
30.30% complete
30.33% complete
30.36% complete
30.38% complete
30.41% complete
30.44% complete
30.47% complete
30.49% complete
30.52% complete
30.55% complete
30.58% complete
30.60% complete
30.63% c

KeyboardInterrupt: 

In [55]:
portfolio

,Portfolio Return
date,
2007-01-03,NaN
2007-01-04,0.162145
2007-01-05,-0.092366
2007-01-08,NaN
2007-01-09,NaN
...,...
2021-05-24,NaN
2021-05-25,NaN
2021-05-26,NaN
